In [10]:
-- USE OF CTE(Common Table Expression)

-- CTE to break down complex query

WITH customer_in_USA AS (
    SELECT 
        customerName, country, state
    FROM 
        customers
    WHERE 
        country = 'USA'
)

SELECT 
    customerName, state
 FROM
    customer_in_USA
 WHERE
    state = 'CA'
 ORDER BY customerName;

(11 row(s) affected)

Total execution time: 00:00:01.008

customerName,state
Boards & Toys Co.,CA
Collectable Mini Designs Co.,CA
Corporate Gift Ideas Co.,CA
"Men 'R' US Retailers, Ltd.",CA
Mini Gifts Distributors Ltd.,CA
Mini Wheels Co.,CA
Signal Collectibles Ltd.,CA
Technics Stores Inc.,CA
The Sharp Gifts Warehouse,CA
Toys4GrownUps.com,CA


In [1]:
use ecom

Commands completed successfully

Total execution time: 00:00:00.003

In [13]:
-- Getting top sales using a CTE

-- We’ll use the orders, orderdetails, and employees from the sample database:

-- The following example uses a CTE to retrieve the top 5 sales representatives based on their total sales in the year 2003:

--  Learn more on https://www.mysqltutorial.org/mysql-basics/mysql-cte/
WITH gettop5slaesRepresentitive AS (
    SELECT 
        c.salesRepEmployeeNumber AS employeeNumber,
        SUM(orDt.quantityOrdered *	orDt.priceEach) AS sales
    FROM 
        orders AS o
    INNER JOIN orderdetails orDt
        ON o.orderNumber = orDt.orderNumber
    INNER JOIN customers c
        ON o.customerNumber = c.customerNumber
    WHERE
        YEAR(shippedDate) = '2003'
        AND status = "shipped"
    GROUP BY c.salesRepEmployeeNumber
    ORDER BY sales DESC
    LIMIT 5
)

SELECT 
    CONCAT(e.lastName,  " ",e.firstName) as fullName,
    e.email,
    e.jobTitle,
    topE.sales
FROM employees AS e
JOIN gettop5slaesRepresentitive AS topE
    ON e.employeeNumber = topE.employeeNumber;

-- How it works.

-- First, define a CTE that retrieves the top 5 employees with their total sales in 2003.
-- Second, join the CTE with the employees table to include the first and last names of the sales representatives.


(5 row(s) affected)

Total execution time: 00:00:01.015

fullName,email,jobTitle,sales
Jennings Leslie,ljennings@classicmodelcars.com,Sales Rep,413219.85
Hernandez Gerard,ghernande@classicmodelcars.com,Sales Rep,295246.44
Castillo Pamela,pcastillo@classicmodelcars.com,Sales Rep,289982.88
Nishi Mami,mnishi@classicmodelcars.com,Sales Rep,267249.40
Bott Larry,lbott@classicmodelcars.com,Sales Rep,261536.95


In [3]:
-- TRANSACTION
-- use ecom;
-- SET autocommit = 0;
-- SET autocommit = 1;

-- By default, when you execute an SQL statement, MySQL automatically wraps it in a transaction and commits the transaction automatically.

-- To instruct MySQL to not start a transaction implicitly and commit the changes automatically, you set the value of the autocommit variable to 0 or OFF:

-- SET autocommit = 1;

-- To enable the auto-commit mode, you set the value of the autocommit variable to 1 or ON:

-- SET autocommit = 1;

CREATE TABLE users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    email VARCHAR(255) UNIQUE
);


Commands completed successfully

Total execution time: 00:00:00.100

In [11]:
-- INSERT INTO TABLE


START TRANSACTION;

INSERT INTO users VALUES(1, 'irfan uddin', "test@gmail.com");

UPDATE users
    SET email = 'tes2t@gmail.com'
WHERE email = 'test@gmail.com';

COMMIT;

Commands completed successfully

Total execution time: 00:00:00.019

: 1062 (23000): Duplicate entry '1' for key 'users.PRIMARY'

In [14]:
SELECT * FROM users;

(1 row(s) affected)

Total execution time: 00:00:01.007

id,name,email
1,irfan uddin,tes2t@gmail.com


In [13]:
-- USE CALL OF ROLLBACK
START TRANSACTION;

INSERT INTO users (id, name) 
VALUES (2, 'jane');

UPDATE users 
SET email = 'jane.doe@example.com' 
WHERE id = 2;

ROLLBACK;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.015

In [15]:
-- Practise Transaction with Stored proc

-- STart with creating  tables, inserting data and create stored proc and executing it 

Total execution time: 00:00:00

In [16]:
-- Creating Account table and Transaction table

START TRANSACTION;
CREATE TABLE accounts (
    account_id INT AUTO_INCREMENT PRIMARY KEY,
    account_name VARCHAR(255) NOT NULL,
    balance DECIMAL(10, 2) NOT NULL
);

CREATE TABLE transactions (
    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
    account_id INT NOT NULL,
    balance DECIMAL(10, 2) NOT NULL,
    transaction_type ENUM('DEPOSIT', 'WITHDRAWAL') NOT NULL,
    FOREIGN KEY(account_id) REFERENCES accounts(account_id)
);

COMMIT;



Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.186

In [31]:
-- INSERT DATA INTO ACCOUNT
-- SET autocommit = 1;
INSERT INTO accounts (account_name, balance)
VALUES ('JONE DOE', 200),
('MIC HESS', 300);
COMMIT;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.023

In [39]:
SELECT account_id FROM accounts WHERE account_id = 3;
SELECT account_id FROM accounts WHERE account_id = 4;


(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:02.015

account_id
3


account_id
4


In [37]:
select * from accounts;
select * from transactions;

(6 row(s) affected)

(0 row(s) affected)

Total execution time: 00:00:02.022

account_id,account_name,balance
3,JONE DOE,200.00
4,MIC HESS,300.00
5,JONE DOE,200.00
6,MIC HESS,300.00
7,JONE DOE,200.00
8,MIC HESS,300.00


transaction_id,account_id,balance,transaction_type


In [33]:
-- CRETAE THE STORED PROC TO TRANSFER BLANCE BETWEEN ACCOUNT
-- USE SQL TRANSACTION TO COMPLETE THE WHOLE OPERATION AND RETURN ERROR OR RESULT

use ecom;

DROP PROCEDURE IF EXISTS TransferBalanceBeteenAccount;

CREATE PROCEDURE TransferBalanceBeteenAccount(
    IN sender_id INT,
    IN receiver_id INT,
    IN amount DECIMAL(10, 2)
)
BEGIN
    DECLARE rollback_message VARCHAR(255) DEFAULT 'Transaction rolled back: Insufficient funds';
    DECLARE success_message VARCHAR(255) DEFAULT 'Transaction successfull';

    START TRANSACTION;

    UPDATE accounts SET balance = balance - amount WHERE account_id = sender_id;
    UPDATE accounts SET balance = balance + amount WHERE account_id = receiver_id;

     
    IF (SELECT balance FROM accounts WHERE account_id = sender_id) < 0 THEN
        ROLLBACK;

        SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = rollback_message;
    ELSE
        INSERT INTO transactions(account_id, balance, transaction_type) VALUES(sender_id, -amount, 'WITHDRAWAL');
        INSERT INTO transactions(account_id, balance, transaction_type) VALUES(receiver_id, amount, 'DEPOSIT');
        
        COMMIT;

        SELECT success_message AS 'result';

    END IF;

END;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.048

In [40]:
-- Execute the STORED PROC
use ecom;
CALL TransferBalanceBeteenAccount(3, 4, 200);

Commands completed successfully

(1 row(s) affected)

Total execution time: 00:00:01.030

result
Transaction successfull


In [43]:
-- CHECK ACCOUNT AND TRANSECTION
use ecom;
SELECT * FROM accounts WHERE account_id IN (3, 4);
SELECT * FROM transactions WHERE account_id IN (3, 4);


Commands completed successfully

(2 row(s) affected)

(2 row(s) affected)

Total execution time: 00:00:02.021

account_id,account_name,balance
3,JONE DOE,0.00
4,MIC HESS,500.00


transaction_id,account_id,balance,transaction_type
6,3,-200.00,WITHDRAWAL
7,4,200.00,DEPOSIT


In [44]:
-- Rerun with some wrong amount
use ecom;
CALL TransferBalanceBeteenAccount(5, 6, 800);

Commands completed successfully

Total execution time: 00:00:00.012

: 1644 (45000): Transaction rolled back: Insufficient funds

In [47]:
-- CHECK ACCOUNT AND TRANSECTION
use ecom;
SELECT * FROM accounts WHERE account_id IN (5, 6);
SELECT * FROM transactions WHERE account_id IN (5, 6);

Commands completed successfully

(2 row(s) affected)

(0 row(s) affected)

Total execution time: 00:00:02.025

account_id,account_name,balance
5,JONE DOE,200.00
6,MIC HESS,300.00


transaction_id,account_id,balance,transaction_type
